In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import argparse
from torchinfo import summary

from code.models.multimodal import MultimodalModel
from code.models.compressor import CompressorModel
from code.models.attention import AttentionModel
from code.dataloader import ChDataModule
from code.train import train_epoch, eval_epoch, train_loop